# Import libraries

In [ ]:
# Pre-requisites. via
# https://github.com/haotian-liu/LLaVA
!git clone https://github.com/haotian-liu/LLaVA.git
%cd LLaVA
!conda create -n llava python=3.10 -y
!conda activate llava
!pip install --upgrade pip  # enable PEP 660 support
!pip install fastapi kaleido python-multipart typing-extensions==4.5.0 uvicorn
!pip install -e .

In [ ]:
import inspect
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
# Import libraries. via
# https://medium.com/supportvectors/comparing-image-to-text-models-blip-blip2-and-llava-ebd66d2eb6d8
import torch
from llava.serve.cli import load_image
from llava.constants import IMAGE_TOKEN_INDEX
from llava.model.builder import load_pretrained_model
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria

# Instantiate model

In [ ]:
# Instantiate the model. via
# https://github.com/haotian-liu/LLaVA
# https://medium.com/supportvectors/comparing-image-to-text-models-blip-blip2-and-llava-ebd66d2eb6d8
model_path = 'liuhaotian/llava-v1.5-13b'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path),
    device=device
)

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory to store inferences

In [ ]:
os.makedirs('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions', exist_ok=True)

# Import datasets

## DISARM

In [ ]:
DISARM_test_all_captioned_Qwen_VL_Chat = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_Qwen-VL-Chat.csv')
DISARM_test_all_captioned_InternLM_XComposer = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_InternLM-XComposer.csv')
DISARM_test_all_captioned_llava = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_llava-v1.5-13b.csv')
DISARM_test_all_captioned_BLIP_2 = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_BLIP-2.csv')

# Set prompts and define a function to call the model

In [ ]:
def prompt_vanilla(entity):
    return inspect.cleandoc(f"""
    Does this meme harmfully target {entity}?
    yes: a social entity is subjected to some form of ill-treatment such as mental abuse, psycho-physiological injury, proprietary damage, emotional disturbance, or public image damage, based on their background (bias, social background, educational background, etc.) by a meme author.
    no: any benign mention (or depiction) of a social entity via humour, limerick, harmless pun or any content that does not cause distress.
    Constraint: Without using any other words, answer either yes or no.""")

In [ ]:
def prompt_with_OCR(entity, OCR):
    return inspect.cleandoc(f"""
    Text on this meme: \"\"\"
    {OCR}
    \"\"\"
    Does this meme harmfully target {entity}?
    yes: a social entity is subjected to some form of ill-treatment such as mental abuse, psycho-physiological injury, proprietary damage, emotional disturbance, or public image damage, based on their background (bias, social background, educational background, etc.) by a meme author.
    no: any benign mention (or depiction) of a social entity via humour, limerick, harmless pun or any content that does not cause distress.
    Constraint: Without using any other words, answer either yes or no.""")

In [ ]:
def prompt_with_caption(entity, caption):
    return inspect.cleandoc(f"""
    Description of this meme: \"\"\"
    {caption}
    \"\"\"
    Does this meme harmfully target {entity}?
    yes: a social entity is subjected to some form of ill-treatment such as mental abuse, psycho-physiological injury, proprietary damage, emotional disturbance, or public image damage, based on their background (bias, social background, educational background, etc.) by a meme author.
    no: any benign mention (or depiction) of a social entity via humour, limerick, harmless pun or any content that does not cause distress.
    Constraint: Without using any other words, answer either yes or no.""")

In [ ]:
# Use llava-v1.5-13b for the inference. via
# https://github.com/haotian-liu/LLaVA/blob/main/llava/serve/cli.py
# https://medium.com/supportvectors/comparing-image-to-text-models-blip-blip2-and-llava-ebd66d2eb6d8
def get_prediction(image, prompt):
    image = load_image(image)
    image_tensor = image_processor([image], return_tensors='pt')['pixel_values']
    image_tensor = image_tensor.to(model.device, dtype=torch.float16)
    prompt = f'USER: <image>\n{prompt}\nASSISTANT:'
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)
    stopping_criteria = KeywordsStoppingCriteria(["</s>"], tokenizer, input_ids)
    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor,
            do_sample=True,
            temperature=0.2,
            max_new_tokens=512,
            use_cache=True,
            stopping_criteria=[stopping_criteria])
    output = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
    return output[:-4]

# Define function to clean responses

In [ ]:
def remap(x):
    x = x.lower()
    if x == 'yes':
        return 'harmful'
    elif x == 'no':
        return 'not harmful'
    elif 'yes' in x and 'no' not in x:
        return 'harmful'
    elif 'no' in x and 'yes' not in x and 'not' not in x:
        return 'not harmful'
    else:
        return None

# Call the `get_prediction` function and save inferences

GROUNDING [ABSENT] & PROMPT [VANILLA]

In [ ]:
DISARM_test_all_grounding_absent_prompt_vanilla = DISARM_test_all_captioned_Qwen_VL_Chat.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_vanilla_images = DISARM_test_all_grounding_absent_prompt_vanilla['image'].values
DISARM_test_all_grounding_absent_prompt_vanilla_entities = DISARM_test_all_grounding_absent_prompt_vanilla['target'].values
DISARM_test_all_grounding_absent_prompt_vanilla['prediction'] = [get_prediction(image, prompt_vanilla(entity)) for image, entity in zip(DISARM_test_all_grounding_absent_prompt_vanilla_images, DISARM_test_all_grounding_absent_prompt_vanilla_entities)]
DISARM_test_all_grounding_absent_prompt_vanilla.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_prompt[VANILLA]_prediction[llava-v1.5-13b].csv', index=False)
DISARM_test_all_grounding_absent_prompt_vanilla['prediction'] = DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_vanilla['prediction'] = DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_vanilla['labels'].values, DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_vanilla['labels'].values, DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].values, labels=['harmful', 'not harmful']))

harmful        595
not harmful     17
Name: prediction, dtype: int64
0
0.3625514215674241
              precision    recall  f1-score   support

     harmful       0.52      0.97      0.67       316
 not harmful       0.47      0.03      0.05       296

    accuracy                           0.51       612
   macro avg       0.49      0.50      0.36       612
weighted avg       0.49      0.51      0.37       612



GROUNDING [ABSENT] & PROMPT [OCR]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_OCR = DISARM_test_all_captioned_Qwen_VL_Chat.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_OCR_images = DISARM_test_all_grounding_absent_prompt_with_OCR['image'].values
DISARM_test_all_grounding_absent_prompt_with_OCR_entities = DISARM_test_all_grounding_absent_prompt_with_OCR['target'].values
DISARM_test_all_grounding_absent_prompt_with_OCR_texts = DISARM_test_all_grounding_absent_prompt_with_OCR['extracted_text'].values
DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'] = [get_prediction(image, prompt_with_OCR(entity, text)) for image, entity, text in zip(DISARM_test_all_grounding_absent_prompt_with_OCR_images, DISARM_test_all_grounding_absent_prompt_with_OCR_entities, DISARM_test_all_grounding_absent_prompt_with_OCR_texts)]
DISARM_test_all_grounding_absent_prompt_with_OCR.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_prompt[OCR]_prediction[llava-v1.5-13b].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'] = DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'] = DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_OCR['labels'].values, DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_OCR['labels'].values, DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].values, labels=['harmful', 'not harmful']))

harmful        600
not harmful     12
Name: prediction, dtype: int64
0
0.35790846707877283
              precision    recall  f1-score   support

     harmful       0.52      0.98      0.68       316
 not harmful       0.50      0.02      0.04       296

    accuracy                           0.52       612
   macro avg       0.51      0.50      0.36       612
weighted avg       0.51      0.52      0.37       612



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [Qwen_VL_Chat]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat = DISARM_test_all_captioned_Qwen_VL_Chat.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[Qwen-VL-Chat]_prompt[CAPTION]_prediction[llava-v1.5-13b].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['harmful', 'not harmful']))

harmful        591
not harmful     21
Name: prediction, dtype: int64
0
0.3720762801762666
              precision    recall  f1-score   support

     harmful       0.52      0.97      0.67       316
 not harmful       0.52      0.04      0.07       296

    accuracy                           0.52       612
   macro avg       0.52      0.50      0.37       612
weighted avg       0.52      0.52      0.38       612



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [InternLM_XComposer]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer = DISARM_test_all_captioned_InternLM_XComposer.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_images = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_entities = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_captions = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_images, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_entities, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[InternLM-XComposer]_prompt[CAPTION]_prediction[llava-v1.5-13b].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['harmful', 'not harmful']))

harmful        594
not harmful     18
Name: prediction, dtype: int64
0
0.366025057744803
              precision    recall  f1-score   support

     harmful       0.52      0.97      0.67       316
 not harmful       0.50      0.03      0.06       296

    accuracy                           0.52       612
   macro avg       0.51      0.50      0.37       612
weighted avg       0.51      0.52      0.38       612



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [llava]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_llava = DISARM_test_all_captioned_llava.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_llava_images = DISARM_test_all_grounding_absent_prompt_with_caption_llava['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_llava_entities = DISARM_test_all_grounding_absent_prompt_with_caption_llava['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_llava_captions = DISARM_test_all_grounding_absent_prompt_with_caption_llava['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_llava_images, DISARM_test_all_grounding_absent_prompt_with_caption_llava_entities, DISARM_test_all_grounding_absent_prompt_with_caption_llava_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_llava.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[llava-v1.5-13b]_prompt[CAPTION]_prediction[llava-v1.5-13b].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_llava['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_llava['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['harmful', 'not harmful']))

harmful        542
not harmful     70
Name: prediction, dtype: int64
0
0.39585005158775655
              precision    recall  f1-score   support

     harmful       0.51      0.87      0.64       316
 not harmful       0.40      0.09      0.15       296

    accuracy                           0.49       612
   macro avg       0.45      0.48      0.40       612
weighted avg       0.45      0.49      0.40       612



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [BLIP-2]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2 = DISARM_test_all_captioned_BLIP_2.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_images = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_entities = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_captions = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_images, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_entities, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[BLIP_2]_prompt[CAPTION]_prediction[llava-v1.5-13b].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['harmful', 'not harmful']))

harmful        582
not harmful     30
Name: prediction, dtype: int64
0
0.4021069315588834
              precision    recall  f1-score   support

     harmful       0.53      0.97      0.68       316
 not harmful       0.67      0.07      0.12       296

    accuracy                           0.53       612
   macro avg       0.60      0.52      0.40       612
weighted avg       0.59      0.53      0.41       612

